In [58]:
# a. set magic 
%load_ext autoreload
%autoreload 2

# b. load packages 
import numpy as np

# c. local modules
from contracept_model import child_model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [106]:
model = child_model()

In [108]:
# Reset the modelameters 
model.mu = -0.12
model.eta2 = 0.17
model.eta3 = -0.05

model.beta = 0.99

# Step 2: Initialize value function and policy
n = model.n
T = model.T
V = np.nan + np.zeros([n, T])
V[:, T - 1] = life(N,model)
pnc1 =  np.nan + np.zeros([n, T])
pnc1[:, T-1] = 1

# Step 3: Backward induction
for t in range(T - 2, -1, -1):
    ev1, pcn = model.bellman(ev0 = V[:, t+1], output=2)
    V[:, t] = ev1  
    pnc1[:, t] = pnc


print(pnc1.round(3))
print(V.round(2))




[[0.444 0.445 0.445 0.446 0.448 0.45  0.452 0.456 0.461 0.468 0.477 0.488
  0.501 0.518 0.536 0.554 0.566 0.568 0.558 1.   ]
 [0.387 0.387 0.388 0.388 0.389 0.389 0.391 0.392 0.395 0.399 0.404 0.411
  0.421 0.435 0.452 0.474 0.499 0.522 0.535 1.   ]
 [0.384 0.384 0.384 0.384 0.384 0.384 0.384 0.385 0.385 0.386 0.388 0.391
  0.395 0.401 0.411 0.425 0.446 0.476 0.511 1.   ]
 [0.435 0.435 0.435 0.435 0.435 0.435 0.435 0.435 0.435 0.435 0.435 0.436
  0.436 0.437 0.439 0.443 0.45  0.462 0.488 1.   ]
 [0.53  0.53  0.53  0.53  0.53  0.53  0.53  0.53  0.53  0.53  0.53  0.53
  0.53  0.53  0.53  0.53  0.53  0.53  0.53  1.   ]]
[[ 9.46  9.1   8.73  8.35  7.97  7.58  7.19  6.78  6.37  5.93  5.49  5.02
   4.52  3.99  3.41  2.79  2.12  1.42  0.7   0.  ]
 [ 9.1   8.73  8.37  7.99  7.62  7.23  6.85  6.45  6.05  5.64  5.22  4.79
   4.34  3.87  3.36  2.82  2.22  1.57  0.86  0.12]
 [ 8.49  8.13  7.76  7.39  7.01  6.63  6.25  5.86  5.47  5.07  4.67  4.26
   3.84  3.4   2.96  2.49  1.99  1.44  0.83  0.14]
